# 딥러닝 강의 요약
## 개요
- 회귀(Regression): 연속적인 데이터를 학습하여 결과를 바탕으로 임의의 데이터 결과 예측
- 분류(Classification): 데이터의 분포를 학습하여 결과를 바탕으로 임의의 데이터가 어느 분포에 속하는지
- 신경망(Neural Network): 인간의 뇌 세포 상호작용 모방
     - 이전 뉴런 출력을 입력으로 받아 + - 가중치를 통해 전체 합하여 특정 입계치(threshold)를 넘으면 다음 뉴런으로 전달하는 원리

## 라이브러리 호출

In [1]:
import numpy as np
import pandas as pd

## 수치 미분
### 미분
- 입력 변수 x가 미세하게 변할 때, 함수 f가 얼마나 변할 수 있는지 근처함수 변화량
- 함수는 입력 변수의 미세한 변화에 얼마나 민감하게 반응 하는지

### 편미분
- 입력 변수가 하나 이상인 다변수 함수에서, 미분하고자 하는 변수 하나를 제외하고 나머지 변수들을 상수 취급해, 해당 변수만 미분
- 각 변수 변화에 따른 변화량

### 연쇄법칙
- 합성함수를 구성하는 각 함수의 미분의 곱으로 합성합수를 미분

### 파이썬으로 수치미분 구현
#### 수치미분 정의 및 예제

In [2]:
def numerical_derivative(f,x): # f==함수, 외부에서 def, lambda정의
    delta_x=1e-4 #lim에 해당되는 작은값 10의 -4승
    return (f(x+delta_x)-f(x-delta_x))/(2*delta_x) #전방차분 f(x+dx)-f(x), 후방차분보다 중앙차분 f(x+dx)-f(x-dx)/2dx이 오차가 적음
                                                    #너무 작은 값일때 오버플로우가 발생하지 않도록 함

In [4]:
def my_func(x):
    return x**2 #f(x)=x^2

In [5]:
result=numerical_derivative(my_func,3)  #f'(3)
print(result)

6.000000000012662


In [6]:
def my_func2(x):
    return 3*x*(np.exp(x)) #3xe^x

In [7]:
result=numerical_derivative(my_func2,2)  #f'(2)
print(result)

66.50150507518049


In [3]:
def numerical_derivative2(f,x): #x 임의의 행렬
    delta_x=1e-4
    grad=np.zeros_like(x) #계산된 수치미분 값 저장 변수
    it = np.nditer(x,flags=['multi_index'], op_flags=['readwrite'])
    #모든 입력 변수에 대해 편미분 하기위한 iterator
    #nider배열 반복해 요소 하나에 접근해 설정에 따라 값을 알려줌
    
    while not it.finished: #변수 개수 만큼
        idx=it.multi_index #행렬에서 가리키는 값
        
        tmp=x[idx] #numpy 타입== mutable (변할 수 있음)이므로 원래 값 보관
        x[idx]=float(tmp)+delta_x #미세하게 변화시킴
        fx1=f(x) #f(x+delta_x)
        
        x[idx]=tmp-delta_x
        fx2=f(x) #f(x-delta_x)
        grad[idx]=(fx1-fx2)/(2*delta_x)
        
        x[idx]=tmp
        it.iternext()  #하나의 변수에 대한 수치미분 계산, 다음 변수로 넘어감
        
    return grad

In [9]:
def func(input_ob):
    x=input_ob[0]
    y=input_ob[1]
    
    return (2*x+3*x*y+np.power(y,3))

In [10]:
input=np.array([1.0,2.0])
numerical_derivative2(func,input)

array([ 8.        , 15.00000001])

## 지도,비지도 학습
### 학습 종류
- 지도학습(supervised): 학습할 데이터의 입력과 입력에 대응되는 정답을 이용해 데이터의 특성과 분포를 학습하고 미래 결과 예측
    - 회귀,분류
- 비지도학습(unsupervised): 학습할 데이터에 정답이 없음, 입력값만 존재, 입력값의 특성과 분포만을 파악
     - 군집화
     
     

### 지도 학습
- Training data == 입력과 정답을 포함
- Test data == 미지의 데이터, 미래 값 예측
- 트레이닝 데이터 학습(상관관계 파악) -> 테스트 데이터 예측 

#### 회귀
- 트레이닝 데이터를 이용해 연속적인 수치 값 예측

#### 분류
- 트레이닝 데이터를 이용해 주어진 입력값이 어떤 종류인지 구별 (범주형)

### 비지도 학습
- 입력 값에 대한 패턴, 특성 등을 학습을 통해 발견

#### 군집화
- 분류와 차이점
    - 분류는 정답에 가장 대응하는 임의의 직선을 찾아 기준으로 삼음
    - 군집화는 정답이 없어, 특성을 찾아 구분

## 선형 회귀(Linear Regression)
### 오차, 손실 함수(loss function)
- training data의 정답(t)와 입력(x)에 대한 계산 값 y의 차이를 모두 더해 수식으로 나타냄
- 각 오차(잔차) 값의 부호가 다르므로(최소 오차값 판별 불가) 합이 아닌 제곱수 이용
    - (t-y)^2 = (t-[Wx+b])^2
    - 양수값 계산해 오차값을 크게 판별
    
- loss function = Σ{(t-y)^2 = (t-[Wx+b])^2}/n = E(W,b)
       - W,b에 영향 받음(training data에서 주어짐)

- 손실함수가 최소 값을 갖도록 (W,b)를 구하는 것이 선형회귀의 최종 목적
    
#### 학습 개념
1. training data 분석
     - 입력(x)에 대해 출력(y) 관계 파악
2. 가중치(기울기 W), 바이어스 b(y 절편) 찾음
    - y=Wx+b

- 오차
    - t- y= t-(Wx+b) , t-정답
    - 오차가 클 경우, 임의로 설정한 가중치와 바이어스 값이 잘못됨
    - 오차가 작을 경우, 미래 값 예측 정확도 상승 예상
    - 오차가 가장 작은 가중치 W, 바이어스 b 찾아야함

#### 경사하강법(gradient decent algorithm)
- 손실함수가 최소값을 갖는 가중치 W와 바이어스 b를 찾는 방법
- 손실 함수(포물선)에 대한 최소값
1. 임의의 가중치 W 선택
2. W 에서의 직선의 기울기를 나타내는 편미분값(E'(W))
    - E'(W) 양수일 때, W 왼쪽으로 이동(감소)
    - E'(W) 음수일 때, W 오른쪽으로 이동(증가)
    - W=W-a(E'(W)) , a=학습율(옵티마이저), w값의 감소 또는 증가 되는 비율/ W<0,a>0
3. 미분값이 작아지는 방향으로 W 감소(or 증가) 시켜나감
4. 기울기가 더 이상 작아지지 않은 곳 찾음(E(W)의 최소값)

## 로지스틱 회귀(Logistic Regression) - Classification

### 분류
 1. 정의
  - Training Data 특성과 관계 등을 파악한 후, 미지의 입력 데에터에 대해 결과가 어떤 종류의 값으로 분류될 수 있는지 예측

### 알고리즘
1. 트레이닝 데이터 특성과 분포를 나타내는 최적의 직선을 찾고(Linear Regression)
  
2. 그 직선을 기준으로 위(1) 또는 아래(0)로 분류 해주는 알고리즘 -> 정확도 높음

(x,t)->Regression->Classification->True or False
           (Wx+b)-> (sigmoid)
             
             
3. 함수 값이 0과 1사이에 머물러있는 Sigmoid 함수 사용
 - 분류 시스템은 출력 값 y가 0 or 1만 가져야 하므로 적합
 - Wx+b=z에서 z가 어떤 값을 갖더라도 출력함수로 출력함수로 Sigmoid사용해 0.5보다 크면 결과로 1이 나올성이 높고, 작으면 0이 나올성 높음
 -> Sigmoid 함수의 실제 계산값 Sigmoid(z)는 결과가 나타날 확률 의미

### 손실함수

$y=\frac{1}{1+e^{-(Wx+b)}} , t_{i}=0  or  1$
  
  일때,

손실함수
$E(W,b)=-\sum_{i=1}^{n} {t_{i}logy_{i}+(1-t_{i})log(1-y_{i})}$
  
  
- 수치미분으로 나타냄  
 $W=W-\alpha \frac{\delta E(W,b)}{\delta W}$
   
 $b=b-\alpha \frac{\delta E(W,b)}{\delta b}$

### 구현
1. 학습을 위한 트레이닝 데이터 준비
 - python slicing or list comprehension등을 이용해 입력 x와 레이블 t를 numpy타입으로 분리(t=0 or 1)
  
  
2. sigmoid 함수에 입력으로 들어가는 임의의 직선z=Wx+b (numpy random으로 W,b)
  
3. 손실함수 구현 
 - def sigmoid(x):
    return 1 / (1+np.exp(-x))
 - def loss_func(x, t):
    
    delta = 1e-7    # 아주작은값 델타, y=0 or 1 일때 log 내 값이 무한대
    
    z = np.dot(x,W) + b #행렬곱 dot
    y = sigmoid(z)
    
  - return  -np.sum( t*np.log(y + delta) + (1-t)*np.log((1 - y)+delta ) )  #손실함수 엔트로피
4. 학습률은 10 ^-3 ~ 10^-5 등
  
5. W,b구하기
  - f = lambda x : loss_func(x_data,t_data)  # f(x) = loss_func(x_data, t_data)
  - W -= learning_rate * numerical_derivative(f, W)
    
    b -= learning_rate * numerical_derivative(f, b)

## XOR 문제
### 논리게이트
1. 논리테이블(AND,OR,NAND,XOR)은 입력데이터 (x1,x2),정답데이터(t=0 or 1)인 머신러닝 트레이닝 데이터와 개념적으로 동일함

2. 손실함수 이용해 Logistic Regression 알고리즘으로 데이터 분류하고 결과 예측 가능

3. 논리게이트 class 정의-> Logistic Regression 검증

## 로지스틱 클래스
```class LogicGate:
    def __init__(self,gate_name,xdata,tdata) #__xdata,__tdata,__W,__b초기화
    def __lost_func(self) #손실함수 cross-entropy
    def error_val(self) # 손실함수 값 계산
    def train(self) #수치미분을 이용해 손실함수 최소값 찾는 method
    def predict(self,xdata) #미래 값 예측 method
   ``` 
## Usage
```xdata=np.array(0,0],0,1],1,0],1,1]]) #입력데이터 생성
tdata=np.array(0,0,0,1]) #정답데이터 생성(AND)

AND_obj=LogicGate("AND_GATE",xdata,tdata) #LogicGate 객체생성

AND_obj.train() #손실함수 최소값 갖도록 학습

AND_obj.predict() #임의의 데이터에 대해 결과 예측
```